 **1. Install the required modules**

In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install tqdm

**2. Import the required modules**

In [1]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [32]:
data = []
for i in tqdm(range(1,51)):

  link = 'https://books.toscrape.com/catalogue/page-'+ str(i)+'.html'
  res = requests.get(link)
  soup = BeautifulSoup(res.text, 'html.parser')

  for sp in soup.find_all('li', class_= 'col-xs-6 col-sm-4 col-md-3 col-lg-3'):

    img_link      = 'https://books.toscrape.com/' + sp.find('img').get('src')
    book_link     = 'https://books.toscrape.com/catalogue/' + sp.find('article', class_='product_pod').find('a').get('href')
    rating        = sp.find('article', class_='product_pod').find('p').get('class')[1]
    title         = sp.find('article', class_='product_pod').find_all('a')[1].get('title')
    price         = sp.find('article', class_='product_pod').find_all('p')[1].text[2:]
    availability  = sp.find('article', class_='product_pod').find_all('p')[2].text.strip()

    data.append([title, rating, availability, price, book_link, img_link])

100%|██████████| 50/50 [00:52<00:00,  1.05s/it]


In [33]:
df_1 = pd.DataFrame(data,columns = ['Title','Rating','Availability','Price_in_euros','Book_link','Image_link'])
df_1

,Title,Rating,Availability,Price_in_euros,Book_link,Image_link
0,A Light in the Attic,Three,In stock,51.77,https://books.toscrape.com/catalogue/a-light-i...,https://books.toscrape.com/../media/cache/2c/d...
1,Tipping the Velvet,One,In stock,53.74,https://books.toscrape.com/catalogue/tipping-t...,https://books.toscrape.com/../media/cache/26/0...
2,Soumission,One,In stock,50.10,https://books.toscrape.com/catalogue/soumissio...,https://books.toscrape.com/../media/cache/3e/e...
3,Sharp Objects,Four,In stock,47.82,https://books.toscrape.com/catalogue/sharp-obj...,https://books.toscrape.com/../media/cache/32/5...
4,Sapiens: A Brief History of Humankind,Five,In stock,54.23,https://books.toscrape.com/catalogue/sapiens-a...,https://books.toscrape.com/../media/cache/be/a...
...,...,...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,One,In stock,55.53,https://books.toscrape.com/catalogue/alice-in-...,https://books.toscrape.com/../media/cache/96/e...
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Four,In stock,57.06,https://books.toscrape.com/catalogue/ajin-demi...,https://books.toscrape.com/../media/cache/09/7...
997,A Spy's Devotion (The Regency Spies of London #1),Five,In stock,16.97,https://books.toscrape.com/catalogue/a-spys-de...,https://books.toscrape.com/../media/cache/1b/5...
998,1st to Die (Women's Murder Club #1),One,In stock,53.98,https://books.toscrape.com/catalogue/1st-to-di...,https://books.toscrape.com/../media/cache/2b/4...


In [35]:
df_1.isnull().sum()

Title             0
Rating            0
Availability      0
Price_in_euros    0
Book_link         0
Image_link        0
dtype: int64

In [36]:
book = []
for i in tqdm(range(0,1000)):

    link = df_1['Book_link'][i]
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')

    category                  = soup.find('ul', class_ = 'breadcrumb').find_all('a')[-1].text
    Description               = soup.find_all('p')[3].text
    UPC                       = soup.find_all('td')[0].text
    Price_excl_tax            = soup.find_all('td')[2].text[2:]
    Price_incl_tax            = soup.find_all('td')[3].text[2:]
    Number_of_books_available = soup.find_all('td')[5].text[10:12]
    book.append([link, category, Description, UPC, Price_excl_tax, Price_incl_tax, Number_of_books_available])

100%|██████████| 1000/1000 [13:59<00:00,  1.19it/s]


In [37]:
df_2 = pd.DataFrame(book,columns = ['link','category','Description','UPC','Price_excl_tax','Price_incl_tax','Number_of_books_available'])
df_2

,link,category,Description,UPC,Price_excl_tax,Price_incl_tax,Number_of_books_available
0,https://books.toscrape.com/catalogue/a-light-i...,Poetry,It's hard to imagine a world without A Light i...,a897fe39b1053632,51.77,51.77,22
1,https://books.toscrape.com/catalogue/tipping-t...,Historical Fiction,"""Erotic and absorbing...Written with starling ...",90fa61229261140a,53.74,53.74,20
2,https://books.toscrape.com/catalogue/soumissio...,Fiction,"Dans une France assez proche de la nÃ´tre, un ...",6957f44c3847a760,50.10,50.10,20
3,https://books.toscrape.com/catalogue/sharp-obj...,Mystery,"WICKED above her hipbone, GIRL across her hear...",e00eb4fd7b871a48,47.82,47.82,20
4,https://books.toscrape.com/catalogue/sapiens-a...,History,From a renowned historian comes a groundbreaki...,4165285e1663650f,54.23,54.23,20
...,...,...,...,...,...,...,...
995,https://books.toscrape.com/catalogue/alice-in-...,Classics,\n\n\n\n\n\n,cd2a2a70dd5d176d,55.53,55.53,1
996,https://books.toscrape.com/catalogue/ajin-demi...,Sequential Art,High school student Kei Nagai is struck dead i...,bfd5e1701c862ac3,57.06,57.06,1
997,https://books.toscrape.com/catalogue/a-spys-de...,Historical Fiction,"In Englandâs Regency era, manners and elegan...",19fec36a1dfb4c16,16.97,16.97,1
998,https://books.toscrape.com/catalogue/1st-to-di...,Mystery,"James Patterson, bestselling author of the Ale...",f684a82adc49f011,53.98,53.98,1


In [38]:
df_2.isnull().sum()

link                         0
category                     0
Description                  0
UPC                          0
Price_excl_tax               0
Price_incl_tax               0
Number_of_books_available    0
dtype: int64

In [39]:
Books = pd.DataFrame()

Books['Title']                  = df_1['Title']
Books['Category']               = df_2['category']
Books['UPC']                    = df_2['UPC']
Books['Price_excl_tax']         = df_2['Price_excl_tax']
Books['Price_incl_tax']         = df_2['Price_incl_tax']
Books['Total_Price']            = df_1['Price_in_euros']
Books['Ratings']                = df_1['Rating']
Books['Availability']           = df_1['Availability']
Books['In_Stock_Books']         = df_2['Number_of_books_available']
Books['Description']            = df_2['Description']
Books['Book_link']              = df_1['Book_link']
Books['Image_link']             = df_1['Image_link']

In [40]:
Books

,Title,Category,UPC,Price_excl_tax,Price_incl_tax,Total_Price,Ratings,Availability,In_Stock_Books,Description,Book_link,Image_link
0,A Light in the Attic,Poetry,a897fe39b1053632,51.77,51.77,51.77,Three,In stock,22,It's hard to imagine a world without A Light i...,https://books.toscrape.com/catalogue/a-light-i...,https://books.toscrape.com/../media/cache/2c/d...
1,Tipping the Velvet,Historical Fiction,90fa61229261140a,53.74,53.74,53.74,One,In stock,20,"""Erotic and absorbing...Written with starling ...",https://books.toscrape.com/catalogue/tipping-t...,https://books.toscrape.com/../media/cache/26/0...
2,Soumission,Fiction,6957f44c3847a760,50.10,50.10,50.10,One,In stock,20,"Dans une France assez proche de la nÃ´tre, un ...",https://books.toscrape.com/catalogue/soumissio...,https://books.toscrape.com/../media/cache/3e/e...
3,Sharp Objects,Mystery,e00eb4fd7b871a48,47.82,47.82,47.82,Four,In stock,20,"WICKED above her hipbone, GIRL across her hear...",https://books.toscrape.com/catalogue/sharp-obj...,https://books.toscrape.com/../media/cache/32/5...
4,Sapiens: A Brief History of Humankind,History,4165285e1663650f,54.23,54.23,54.23,Five,In stock,20,From a renowned historian comes a groundbreaki...,https://books.toscrape.com/catalogue/sapiens-a...,https://books.toscrape.com/../media/cache/be/a...
...,...,...,...,...,...,...,...,...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,Classics,cd2a2a70dd5d176d,55.53,55.53,55.53,One,In stock,1,\n\n\n\n\n\n,https://books.toscrape.com/catalogue/alice-in-...,https://books.toscrape.com/../media/cache/96/e...
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Sequential Art,bfd5e1701c862ac3,57.06,57.06,57.06,Four,In stock,1,High school student Kei Nagai is struck dead i...,https://books.toscrape.com/catalogue/ajin-demi...,https://books.toscrape.com/../media/cache/09/7...
997,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,19fec36a1dfb4c16,16.97,16.97,16.97,Five,In stock,1,"In Englandâs Regency era, manners and elegan...",https://books.toscrape.com/catalogue/a-spys-de...,https://books.toscrape.com/../media/cache/1b/5...
998,1st to Die (Women's Murder Club #1),Mystery,f684a82adc49f011,53.98,53.98,53.98,One,In stock,1,"James Patterson, bestselling author of the Ale...",https://books.toscrape.com/catalogue/1st-to-di...,https://books.toscrape.com/../media/cache/2b/4...
